In [1]:
import torch
with open('../data/p1ch4/jane-austen/1342-0.txt', encoding='utf8') as f:
    text = f.read()

先将文本分割成行，然后选择任意一行进行探讨：

In [4]:
lines = text.split('\n')
line = lines[200]
line, len(line)

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him', 70)

创建一个张量，它能够容纳整行字符的独热编码的总数：

In [5]:
letter_t = torch.zeros(len(line), 128)
letter_t.shape

torch.Size([70, 128])

letter_t 用于保存每行字符的独热编码。现在我们只需要在每一行的正确位置上设置一个 1，以便每一行都代表正确的字符，必须设置为 1 的索引对应编码中字符的索引：

In [6]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0 # ord(letter)函数用于获取字符的ASCII码值
    letter_t[i][letter_index] = 1 # 将letter_t列表中的特定位置设置为1，具体位置由i和letter_index决定

定义`clean_words()`函数，它接收文本并以小写字符形式返回，同时去掉标点符号。当我们调用该函数并传入文本行“Impossible, Mr. Bennet”时，我们得到以下内容：

In [7]:
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n', ' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

在编码中建立一个单词到索引的映射：

In [10]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}
len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

注意， word2index_dict 现在是一个字典，单词为键，整数为值。我们将使用它来高效地找到一个单词的索引，因为我们对它进行了独热编码。现在让我们关注我们的句子：我们把它分解成单词，然后进行独热编码。也就是说，我们用每个单词的一个独热编码的向量填充张量。我们创建一个空向量，并为句子中的单词分配一个独热编码的值：

In [11]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])


此时，张量表示一个长度为 11 的句子，编码空间大小为 7261（字典中的单词数）。